![alternatvie text](https://www.doc.zuv.fau.de//M/FAU-Logo/01_FAU_Kernmarke/Web/FAU_Kernmarke_Q_RGB_blue.svg)


# Assignment 4: Self-Supervised Learning 

In the lecture you have learned the main concept of Self-Supervised Learning, that is to derive supervision from proxy / surrogate tasks defined over the data. Over the past years, several proxy tasks have been developed to improve the performance of deep neural networks on downstream tasks. There are two main steps inolved in the self-supervision procedure. First, pre-training the backbone model on the pretext task. Next, the backbone model is finetuned on the downstream task.

In this assignment, you'll work on the task of microscopy image cell segmentation. You're required to implement a proxy task called "Pseudo-label microscopy image cell segmentation". This proxy task pre-trains a cell segmentation network using pseudo-cell segmentation maps extracted via classical computer vision operations. The below figure illustrates the training procedure.

<center><img src="pretrain.png"/></center>

<center><img src="finetune.png"/></center>

You'll need to install the following libraries:
- PyTorch
- Torchvision
- OpenCV
- Matplotlib

## 1. Data Preparation

First, you're required to download the microscopy image dataset, which you'll work with throughout this assignment. You can use this [link](https://github.com/unidesigner/groundtruth-drosophila-vnc) for download. The task involved in the dataset is binary cell segmentation, where the cells of interest are mitochondria. The microscopy images could be found in the directory "stack1/raw/" , while "stack1/mitochondria/" contains the corresponding binary cell segmentation groundtruth.

The dataset contains in total 20 images of resolution 1024x1024. It would be difficult to train on full resolution images, due to the high resolution. Alternatively, this issue could be solve by training with image crops. Note, the groundtruth should also be cropped at the same crop window location. The below class implements RandomCrop. Use a crop size of 256x256.

In [ ]:
import torch
import random
import torchvision.transforms.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.init as init
from PIL import Image
import os
import cv2
import numpy as np


In [ ]:
class RandomCrop(object):

    def __init__(self, size, padding=None, pad_if_needed=False, fill=0, padding_mode='constant'):

        self.size = size
        self.padding = padding
        self.pad_if_needed = pad_if_needed
        self.fill = fill
        self.padding_mode = padding_mode

    @staticmethod
    def get_params(img, output_size):

        w, h = img.squeeze().size()
        th, tw = output_size
        if w == tw and h == th:
            return 0, 0, h, w

        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw

    def __call__(self, data):

        img, mask = data["image"], data["label"]
        # pad the width if needed
        if self.pad_if_needed and img.shape[0] < self.size[1]:
            img = F.pad(img, (self.size[1] - img.shape[0], 0), self.fill, self.padding_mode)
            mask = F.pad(mask, (self.size[1] - mask.shape[0], 0), self.fill, self.padding_mode)
        # pad the height if needed
        if self.pad_if_needed and img.shape[1] < self.size[0]:
            img = F.pad(img, (0, self.size[0] - img.shape[1]), self.fill, self.padding_mode)
            mask = F.pad(mask, (0, self.size[0] - mask.shape[1]), self.fill, self.padding_mode)
        done = False
        while not done:
            i, j, h, w = self.get_params(img, self.size)
            crop_image = F.crop(img, i, j, h, w)
            crop_mask = F.crop(mask, i, j, h, w)
            if (crop_mask==1).sum(dim=0).sum().item() > int(0.1*h*w):
                return crop_image,crop_mask

Next, you will have to implement your own custom dataset to access the images during training and testing. The dataset will be split into 14 images for training and the rest for testing. You can pick the first 14 images for training, while use the rest for testing.

In [1]:
class CustomDataset(Dataset):
    def __init__(self):
        pass
    def __len__(self):
        pass
    def __getitem__(self):
        pass

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## 2. Visualize Data

Visualize some of the microscopy data i.e. image and groundtruth without cropping.

## 3. Supervised Learning (Learning from scratch)

In this part, you will train a fully convolutional regression network (FCRN) with randomly initialized weights and full supervision. Moreover, you should test the trained model on the test set. Use an Adam optimizer with 0.001 learning rate, train for 100 epochs and set the batch size to 14. Furthemore, you should use nn.BCEwithLogitsLoss(). Note since the number of pixels annotated as a foreground is much less than the background you should use weighted BCEloss i.e. nn.BCEwithLogitsLoss(weight=calc_weights(labels)).



In [ ]:
def conv_bn_relu(in_channels, out_channels, kernel_size,affine=False):
    layer = []
    layer.append(nn.Conv2d(in_channels, out_channels, kernel_size, padding=1, bias=False))
    layer.append(nn.BatchNorm2d(out_channels,affine=affine))
    layer.append(nn.ReLU(inplace=True))
    return nn.Sequential(*layer)


def conv_bn_relu_transpose(in_channels, out_channels, kernel_size,affine=False):
    layer = []
    layer.append(nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=2, bias=False))
    layer.append(nn.BatchNorm2d(out_channels,affine=affine))
    layer.append(nn.ReLU(inplace=True))
    return nn.Sequential(*layer)

class FCRN(nn.Module):
    def __init__(self,in_channels=1, out_channels=32, kernel_size=3,affine=True):

        super(FCRN, self).__init__()
        # Encoder
        self.conv1 = conv_bn_relu(in_channels, out_channels, kernel_size,affine=affine)
        self.conv2 = conv_bn_relu(out_channels, out_channels * 2, kernel_size,affine=affine)
        self.conv3 = conv_bn_relu(out_channels * 2, out_channels * 4, kernel_size,affine=affine)

        self.maxpool = nn.MaxPool2d(2, 2)

        # LatentSpace
        self.conv4 = conv_bn_relu(out_channels * 4, out_channels * 16, kernel_size,affine=affine)

        # Decoder
        self.conv5 = conv_bn_relu_transpose(out_channels * 16, out_channels * 4, 2,affine=affine)
        self.conv6 = conv_bn_relu_transpose(out_channels * 4, out_channels * 2, 2,affine=affine)
        self.conv7 = conv_bn_relu_transpose(out_channels * 2, out_channels, 2,affine=affine)
        self.conv8 = nn.Conv2d(out_channels, in_channels, 3, padding=1)


        self._initialize_weights()
    def forward(self, x):
        x = self.maxpool(self.conv1(x))
        x = self.maxpool(self.conv2(x))
        x = self.maxpool(self.conv3(x))

        x = self.conv4(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        out = self.conv8(x)
        return out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.normal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            if isinstance(m,nn.ConvTranspose2d):
                init.normal_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

            elif isinstance(m, nn.BatchNorm2d):
                if m.affine:
                    init.constant_(m.weight, 0.1)
                    init.constant_(m.bias, 0)

In [ ]:
def calc_weights(labels):
    pos_tensor = torch.ones_like(labels)

    for label_idx in range(0, labels.size(0)):
        pos_weight = torch.sum(labels[label_idx] == 1)
        neg_weight = torch.sum(labels[label_idx] == 0)
        ratio = float(neg_weight.item() / pos_weight.item())
        pos_tensor[label_idx] = ratio * pos_tensor[label_idx]

    return pos_tensor

To assess the segmentation performance, we will use the below function which calculate the mean intersection over union. For more information about the metric you can refer to this [blog](https://hasty.ai/docs/mp-wiki/metrics/iou-intersection-over-union#:~:text=To%20define%20the%20term%2C%20in,matches%20the%20ground%20truth%20data.).

In [ ]:
def intersection_over_union(tensor, labels, device=torch.device("cpu")):
    iou = 0
    foreground_acc = 0

    labels_tens = labels.type(torch.BoolTensor)
    ones_tens = torch.ones_like(tensor, device=device)
    zeros_tens = torch.zeros_like(tensor, device=device)
    if tensor.shape[0] > 1:
        temp_tens = torch.where(tensor >= 0.5, ones_tens, zeros_tens)
        intersection_tens = (temp_tens.squeeze().type(torch.BoolTensor) & labels_tens.squeeze()).float().sum((1, 2))

        union_tens = (temp_tens.squeeze().type(torch.BoolTensor) | labels_tens.squeeze()).float().sum((1, 2))
        iou += torch.mean((intersection_tens + 0.0001) / (union_tens + 0.0001))
        foreground_acc += intersection_tens
    else:
        temp_tens = torch.where(tensor >= 0.5, ones_tens, zeros_tens)
        intersection_tens = (temp_tens.squeeze().type(torch.BoolTensor) & labels_tens.squeeze()).float().sum()
        union_tens = (temp_tens.squeeze().type(torch.BoolTensor) | labels_tens.squeeze()).float().sum()
        iou += torch.sum((intersection_tens + 0.0001) / (union_tens + 0.0001))
        foreground_acc += intersection_tens

    total_iou = iou
    return total_iou

Plot the train loss and the train mIoU. You should get a test mIoU of about 45%.

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

## Implement here your training and test functions

## 4. Self-Supervised Learning

Here you will implement a function which extracts pseudo-cell segmentation maps to train your model on the pretext task. You will use the images of the training set to extract pseudo-labels and you should use the pseudo-labels as your target when calculating the loss. Modify your custom dataset class to extract pseudo-labels during pre-training.

To extract pseudo-labels, you can use OpenCV and follow these steps:
- Convert Image to GrayScale cv2.cvtColor().
- Apply histogram equalization i.e. cv2.equalizeHist().
- Apply binary inversion threshold i.e. cv2.THRESH_BIN_INV, use threshold range of [140, 170].
- Convert all values greater than 0 to 255 and 0 otherwise.

In [ ]:
def extract_pseudo_labels():
    pass

Afterwards, you will pre-train your model on the pseudo-label cell segmentation learning with same optimizer, batch, and loss function. However, pretrain only for 50 epochs. Then you will fine-tune the entire pretrained model (encoder and decoder) for 100 epochs assuming you have now access to the groundtruth of the data. You should notice an improvement in the test mIoU of about 10%. 

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# First pretrain

# Fine tune entire model

# test model